In [1]:

import os
import pathlib

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [2]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [3]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [13]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder

def load_object_detection_model(checkpoint_path, pipeline_config_path):
    # Load the pipeline config
    configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)

    # Build the detection model
    detection_model = model_builder.build(model_config=configs['model'], is_training=False)

    # Restore the checkpoint
    ckpt = tf.train.Checkpoint(model=detection_model)
    ckpt.restore(checkpoint_path).expect_partial()

    return detection_model

# Example usage
checkpoint_path = r"C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\after_5000_steps\ckpt-2.0002.index"  # Replace with your actual path
pipeline_config_path = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\pipeline.config'  # Replace with your actual path

loaded_model = load_object_detection_model(checkpoint_path, pipeline_config_path)


In [18]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.builders import model_builder

def load_object_detection_model(checkpoint_path, pipeline_config_path):
    # Load the pipeline config
    configs = config_util.get_configs_from_pipeline_file(pipeline_config_path)

    # Build the detection model
    detection_model = model_builder.build(model_config=configs['model'], is_training=False)

    # Restore the checkpoint
    ckpt = tf.train.Checkpoint(model=detection_model)
    ckpt.restore(checkpoint_path).expect_partial()

    return detection_model

def save_model_to_h5(model, output_h5_path):
    # Convert the Object Detection API model to a Keras model
    keras_model = model._build_model_with_keras()

    # Create a dummy input to save the model architecture
    dummy_input = tf.constant(np.random.rand(1, height, width, channels).astype(np.float32))

    # Perform a forward pass to create the model architecture
    _ = keras_model(dummy_input)

    # Save the entire model to the HDF5 file using Keras save_model function
    tf.keras.models.save_model(keras_model, output_h5_path, include_optimizer=False)

# Example usage
checkpoint_path = r"C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\after_5000_steps\ckpt-2.0002.index"  # Replace with your actual path
pipeline_config_path = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\pipeline.config'  # Replace with your actual path
output_h5_path = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\after_5000_steps\model.h5'  # Replace with your desired output path

loaded_model = load_object_detection_model(checkpoint_path, pipeline_config_path)
save_model_to_h5(loaded_model, output_h5_path)


AttributeError: 'SSDMetaArch' object has no attribute '_build_model_with_keras'

In [15]:
import tensorflow as tf
import numpy as np
from PIL import Image

def check_model_loaded(model):
    # Create a sample input (you may need to adjust this based on your model's input requirements)
    sample_input = np.random.rand(1, height, width, channels).astype(np.float32)

    # Perform inference
    detections = model(sample_input)

    # Check if the output is non-empty (indicating a successful inference)
    if detections['num_detections'][0] > 0:
        print("Model loaded successfully.")
    else:
        print("There might be an issue with the loaded model.")

# Replace with your actual input dimensions (height, width, channels)
height, width, channels = 256, 256, 3

# Example usage
check_model_loaded(loaded_model)


Model loaded successfully.


In [6]:


# Load pipeline config and build a detection model
CONFIG_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\pipeline.config'
CHECKPOINT_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\output'

configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections


AttributeError: 'SSDMetaArch' object has no attribute 'save_weights'

In [ ]:
import cv2
import numpy as np

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\Train\hand-sign_label_map.pbtxt')

In [2]:
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

NameError: name 'cv2' is not defined

In [23]:
print("Input Tensor Shape:", input_tensor.shape)
print("Input Tensor Content:", input_tensor.numpy())


Input Tensor Shape: (1,)
Input Tensor Content: [nan]


In [ ]:
while True: 
    ret, frame = cap.read()
    image_np = np.array(frame)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    input_tensor = tf.convert_to_tensor((image_np_expanded), dtype=tf.float32)

    detections = detect_fn(input_tensor)
    
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.6,
                agnostic_mode=False)

    cv2.imshow('object detection',  cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        break

AttributeError: 'NoneType' object has no attribute 'ndim'

In [ ]:
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder
from streamlit_webrtc import VideoTransformerBase, webrtc_streamer
import os

# Add the path to your pipeline.config and checkpoint folder
CONFIG_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\pipeline.config'
CHECKPOINT_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\output'

# Add the path to your label map file
LABEL_MAP_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\Train\hand-sign_label_map.pbtxt'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
try:
    ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", e)

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections



category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)

class ObjectDetectionTransformer(VideoTransformerBase):
    def __init__(self):
        self.detection_model = detection_model
        self.category_index = category_index

    def transform(self, frame):
        image_np = np.array(frame)
        image_np_expanded = np.expand_dims(image_np, axis=0)

        input_tensor = tf.convert_to_tensor(image_np_expanded, dtype=tf.float32)

        detections = detect_fn(input_tensor)

        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes'] + label_id_offset,
            detections['detection_scores'],
            self.category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=0.6,
            agnostic_mode=False
        )
        print("Detection Scores:", detections['detection_scores'])
        print("Detection Classes:", detections['detection_classes'])

        return cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)

def main():
    st.title("Object Detection with Streamlit and Webcam")

    webrtc_ctx = webrtc_streamer(
        key="example",
        video_transformer_factory=ObjectDetectionTransformer,
        async_transform=True
    )

if __name__ == "__main__":
    main()


In [ ]:
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder
from streamlit_webrtc import VideoTransformerBase, webrtc_streamer
import os

# Add the path to your pipeline.config and checkpoint folder
CONFIG_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\pipeline.config'
CHECKPOINT_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\output'

# Add the path to your label map file
LABEL_MAP_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\Train\hand-sign_label_map.pbtxt'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
try:
    ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", e)
    st.error("Error loading the model. Check the console for details.")

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)


input_type = st.sidebar.selectbox("Select Input Type", ["Select", "Webcam", "Image", "Video"], index=0)

# Initialize video capture based on user choice
if input_type == "Webcam":
    cap = cv2.VideoCapture(0)
elif input_type == "Image":
    uploaded_file = st.sidebar.file_uploader("Choose an image...", type="jpg")
    if uploaded_file is not None:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, 1)
else:  # Video
    uploaded_file = st.sidebar.file_uploader("Choose a video...", type="mp4")
    if uploaded_file is not None:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        video = cv2.VideoCapture()
        video.open(uploaded_file.name)

while True:
    if input_type == "Select":
        st.warning("Please select an input type from the sidebar.")
        break
    elif input_type == "Webcam":
        ret, frame = cap.read()
        if not ret:
            print("Error reading from webcam.")
            break
    elif input_type == "Image":
        if uploaded_file is not None:
            frame = img.copy()
        else:
            break
    else:  # Video
        if uploaded_file is not None:
            ret, frame = video.read()
            if not ret:
                print("Error reading from video.")
                break
        else:
            break
# Setup capture
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

while True:
    ret, frame = cap.read()

    if not ret:
        print("Error reading from webcam.")
        break

    image_np = np.array(frame)
    image_np_expanded = np.expand_dims(image_np, axis=0)

    input_tensor = tf.convert_to_tensor(image_np_expanded, dtype=tf.float32)

    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.6,
        agnostic_mode=False)

    cv2.imshow('object detection', cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break


In [ ]:
import streamlit as st
import cv2
import numpy as np
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import config_util
from object_detection.builders import model_builder
from streamlit_webrtc import VideoTransformerBase, webrtc_streamer
import os

# Add the path to your pipeline.config and checkpoint folder
CONFIG_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\pipeline.config'
CHECKPOINT_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\output'

# Add the path to your label map file
LABEL_MAP_PATH = r'C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection\Train\hand-sign_label_map.pbtxt'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
try:
    ckpt.restore(os.path.join(CHECKPOINT_PATH, 'ckpt-6')).expect_partial()
    print("Model loaded successfully.")
except Exception as e:
    print("Error loading the model:", e)
    st.error("Error loading the model. Check the console for details.")

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH)


def detect_objects(image_np):
    image_np_expanded = np.expand_dims(image_np, axis=0)
    input_tensor = tf.convert_to_tensor(image_np_expanded, dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=0.6,
        agnostic_mode=False)

    # Convert color space from BGR to RGB
    image_np_with_detections = cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB)

    return image_np_with_detections



def main():
    st.title("Object Detection with Streamlit")
    input_types = ["Select", "Webcam", "Image", "Video"]
    input_type = st.sidebar.selectbox("Select Input Type", input_types, index=0)

    if input_type == "Select":
        st.warning("Please select an input type from the sidebar.")
    elif input_type == "Webcam":
        st.warning("To stop the webcam, press 'q' key.")
        cap = cv2.VideoCapture(0)
        while True:
            ret, frame = cap.read()
            if not ret:
                st.error("Error reading from webcam.")
                break
            image = detect_objects(frame)
            st.image(image, channels="RGB")
    elif input_type == "Image":
        uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])
        if uploaded_file is not None:
            file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
            img = cv2.imdecode(file_bytes, 1)
            image = detect_objects(img)
            st.image(image, channels="RGB", use_column_width=True)
    elif input_type == "Video":
        uploaded_file = st.file_uploader("Choose a video...", type=["mp4"])
        if uploaded_file is not None:
            file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
            video = cv2.VideoCapture()
            video.open(uploaded_file.name)
            while True:
                ret, frame = video.read()
                if not ret:
                    st.error("Error reading from video.")
                    break
                image = detect_objects(frame)
                st.image(image, channels="RGB")
    else:
        st.warning("Please select a valid input type from the sidebar.")


if __name__ == "__main__":
    main()


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

NameError: name 'take_photo' is not defined

'C:\\Users\\shubh\\OneDrive\\Desktop\\ML Models\\Tensorflow'

C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow\hand_sign_detection


C:\Users\shubh\OneDrive\Desktop\ML Models\Tensorflow
